In [1]:
from bs4 import BeautifulSoup as bs
import grequests
import requests
import pandas as pd

In [2]:
imo = requests.get('https://www.imovirtual.com/arrendar/apartamento/') # request http
raw_html = imo.text # convert webpage code into raw text
soup = bs(raw_html) # soupify page text

# find number of pages
for i in soup.find_all('ul', class_='pager'):
    pages = int(i.text.split()[-1])
    
# initialize url list with first page for subsequent appending 
urls = ['https://www.imovirtual.com/arrendar/apartamento/'] 

# append further pages links
for page in range(2,pages+1):
    urls.append('https://www.imovirtual.com/arrendar/apartamento/?page=' + str(page))
    
# initialize lists for house prices, types, locations and sizes
prices = []
types = []
location = []
sizes = [] 

for u in urls:
    imo = requests.get(u) # request http
    raw_html = imo.text # convert webpage code into raw text
    soup = bs(raw_html) # soupify page text

    # Find the desired data: price, location (concelho), typology and size (m2)

    # price
    for price in soup.find_all('li', class_="offer-item-price"): 
        prices.append(price.string.split('€')[0].replace(' ','').strip())

    #type 
    for ty in soup.find_all('li', class_="offer-item-rooms hidden-xs"):
        types.append(ty.string)

    #location
    for loc in soup.find_all('p', class_="text-nowrap"):
        location.append(loc.text.split('Apartamento para arrendar: ')[1])

    #size (m2)
    for size in soup.find_all('li', class_="hidden-xs offer-item-area"):
        sizes.append(size.next.split(' ')[0])
        
all_houses = [] # list where all houses will be
all_houses.append([prices, types, location, sizes])
columns = ['price','type','location','size'] # columns names for the dataframe
all_data = pd.DataFrame(all_houses[0]) # create datafame with the list created
all_data = all_data.transpose() # transpose dataframe
all_data.columns = columns # rename dataframe columns
all_data

,price,type,location,size
0,1000,T2,"Cascais e Estoril, Cascais, Lisboa",80
1,1400,T3,"Aldoar, Foz do Douro e Nevogilde, Porto","121,70"
2,680,T2,"Caparica e Trafaria, Almada, Setúbal",74
3,850,T1,"Carcavelos e Parede, Cascais, Lisboa",53
4,830,T3,"Cedofeita, Santo Ildefonso, Sé, Miragaia, São ...",135
...,...,...,...,...
11995,700,T0,"Avenidas Novas, Lisboa",21
11996,550,T1,"Campo de Ourique, Lisboa",45
11997,450,T3,"Alvorninha, Caldas da Rainha, Leiria",140
11998,590,T2,"Algueirão-Mem Martins, Sintra, Lisboa",75


In [4]:
all_data.to_csv('house_data.csv')